# 1

# 利用Keras提供的API建立模型

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
# 匯入Keras的mnist模組
from tensorflow.keras.datasets import mnist
(train_Data, train_Label), (test_Data, test_Label) = mnist.load_data()

model = Sequential([
    layers.Flatten(input_shape=(28, 28)),   # 將輸入資料從 28x28 攤平成 784，另攤平不用計算權重
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax') # output 為 10 個 class
])

# model 每層定義好後需要經過 compile
# sparse_categorical_crossentropy 的標籤是 integer
model.compile(optimizer='Adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 訓練：將建立好的 model 去 fit 我們的 training data
model.fit(train_Data, train_Label, epochs=10)
# 評估：利用 test_Data 去進行模型評估
# verbose = 2 為每個 epoch 輸出一行紀錄
model.evaluate(test_Data, test_Label, verbose=2)

# compile函數

法一：傳遞預定優化器名稱至compile函數，在此情形下，優化器的參數將使用默認值。

In [ ]:
model.compile(optimizer='sgd', loss='mse', metrics=['acc'])

法二：初始化一個優化器對象，然後傳入該函數

In [ ]:
from tensorflow.keras import optimizers            # 匯入優化模組
sgd = optimizers.SGD(lr = 0.01, momentum = 0.9)    # 先設定優化器名稱
model.compile(optimizer = sgd,                     # 選擇優化器：指定優化函式，可直接輸入sgd
              loss='mse',                          # 設定損失函數
              metrics=['acc'])                     # 設定成效衡量指標

法三：或者也可以直接把優化函數代入

In [ ]:
sgd = optimizers.SGD(lr = 0.01, momentum = 0.9)
model.compile(optimizer = optimizers.SGD(lr = 0.01, momentum = 0.9),
              loss='mse', metrics=['acc'])

# metrics

方法一：直接使用字串

In [ ]:
model.compile(optimizer = 'sgd',
              loss='mean_squared_error',
              metrics=['mae', 'acc'])

方法二：使用 tf.keras.metrics 下的類別

In [ ]:
from tensorflow.keras import metrics

model.compile(optimizer = 'sgd',
              loss='mean_squared_error',
              metrics=[metrics.mae, metrics.categorical_accuracy])

# 2

改寫程式CH8_1內model.compile()函式內的 metrics 參數

In [ ]:
from tensorflow.keras import metrics
# model 每層定義好後需要經過 compile
# sparse_categorical_crossentropy 的標籤是 integer
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['acc','mse',metrics.sparse_categorical_crossentropy])

# 3

# 載入資料庫

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import numpy as np

# 加載，預處理數據集
dataset = np.loadtxt("pima-indians-diabetes.csv", delimiter=",")
# 前8個為資料集，第9個為標籤
data = dataset[:, 0:8]    # 資料集
label = dataset[:, 8]     # 標籤

print("data.shape : ", data.shape)   # 印出資料集的維度
print("label.shape : ",label.shape)  # 印出標籤維度

# 創造網路模型

In [ ]:
model = Sequential()
# add加入資料
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # 第3層 結果：有沒有得到糖尿病 / sigmoid介於0~1

print(model.summary())  # 印出網路資訊

# 編譯與訓練模型

In [ ]:
# 編譯模型
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 分成訓練集、驗證集(用於告訴模型何時可以停止)、測試集
# 驗證集不影響權重，只有訓練集會影響權重
# 訓練模型   迭代100次、批處理大小為10,
history = model.fit(data, label, epochs=100, batch_size=10,
                    validation_split = 0.2,    # 劃分資料集的20%作為驗證集用(真正測試只有80%)
                    verbose = 2)               # 印出為精簡模式
print("history: ",history.history)             # 印出歷史紀錄

# 模型評估與預測模型

In [ ]:
# 評估模型
loss, accuracy = model.evaluate(data, label)
print("\nLoss: %.2f, Accuracy: %.2f%%" % (loss, accuracy*100))
# 數據預測
probabilities = model.predict(data)
# 將 probabilities 的輸出值透過np.round()做四捨五入
predictions = [float(np.round(x)) for x in probabilities]
# 計算預測結果跟真實結果的平均差距
accuracy = np.mean(predictions == label)
print("Prediction Accuracy: %.2f%%" % (accuracy*100))

# 4

# Early stopping(提前停止)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import metrics
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

# 匯入Keras 的 mnist模組
from tensorflow.keras.datasets import mnist
(train_Data, train_Label), (test_Data, test_Label) = mnist.load_data()

model = Sequential([
    layers.Flatten(input_shape=(28, 28)),   # 將輸入資料從 28x28 攤平成 784
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # output 為 10 個 class
])

In [ ]:
# 定義訓練的步驟數目
NUM_EPOCHS = 100
# model 每層定義好後需要經過 compile
model.compile(optimizer='adam',
              loss= 'sparse_categorical_crossentropy',
              metrics=['acc',metrics.mse,
                       metrics.sparse_top_k_categorical_accuracy])

In [ ]:
# 定義 tf.keras.EarlyStopping 回調函數
# 並指名監控的對象 => val_sparse_top_k_categorical_accuracy
earlystop_callback = EarlyStopping(
  monitor='val_sparse_top_k_categorical_accuracy', min_delta=0.001,
  patience=1, verbose=1, mode='auto')  
# min_delta=0.001 => 與前一次差異不到0.001就停止
# patience 平常不會寫1，訓練時不應該這樣做
# verbose 秀訊息 
# mode 看目前的監看對象是誰(可設定min, max, auto)

# 將建立好的 model 去 fit 我們的 training data
model.fit(train_Data, train_Label,
          validation_split = 0.2,    # 劃分資料集的 20% 作為驗證集用
          epochs=NUM_EPOCHS,
					callbacks=[earlystop_callback])
# 利用 test_Data 去進行模型評估
# verbose = 2 為每個 epoch 輸出一行紀錄
model.evaluate(test_Data, test_Label, verbose=2)

# sparse_top_k_categorical_accuracy

In [ ]:
import tensorflow as tf
import numpy as np
y_true = [2, 1, 1, 0]  # 假設四個樣本
y_pred = [[0.3, 0.6, 0.1], [0.5, 0.4, 0.1], [0.3, 0.6, 0.1], [0.9, 0., 0.1]]
m = tf.keras.metrics.sparse_top_k_categorical_accuracy(y_true, y_pred, k=2)
print(m.numpy())

# 5

使用Fashion MNIST來當範例

# 載入資料

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

(train_image,train_label),(test_image,test_label)=\
    tf.keras.datasets.fashion_mnist.load_data()
print("train_image : ",train_image.shape)
print("train_label : ",train_label.shape)
print("test_image : ",test_image.shape)
print("test_label : ",test_label.shape)

# 將資料集中的前面9筆資料圖片印出

In [ ]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
              'Sandal', 'Shirt', 'Sneaker','Bag', 'Ankle boot']
# 顯示指定的影像 (這裡顯示九張)
def ShowImage(x,y):
    for i in range(9):
        plt.subplot(330 + 1 + i)
        plt.imshow(x[i], cmap=plt.get_cmap('gray'))
        plt.xticks([])
        plt.yticks([])
        plt.xlabel(class_names[y[i]])
    plt.show()

ShowImage(train_image,train_label)

# 資料初始化與建立模型

In [ ]:
# 對資料集做一個前置處理, 將資料正規到 0~1 之間 # 好處為把數值集中(不那麼分散)
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x,y

# 建立模型
def build_model():
    # 線性疊加
    model = tf.keras.models.Sequential()
    # 改變平坦輸入
    model.add(tf.keras.layers.Flatten(input_shape=(28, 28)))
    # 第一層隱藏層, 包含256個神經元
    model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
    # 第二層隱藏層, 包含128個神經元
    model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
    # 第三層隱藏層, 包含256個神經元
    model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
    # 第四層為輸出層分 10 個類別
    model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))
    return model

model = build_model()
print(model.summary())

# 編譯與訓練模型

In [ ]:
# 編譯模型 complie用於指定三個項目定義
model.compile(optimizer= tf.keras.optimizers.Adam(),    # 優化函式
              loss='sparse_categorical_crossentropy',   # 損失函式
              metrics=['accuracy'])                     # 監控對象

train_images, train_labels = preprocess(train_image, train_label)
batchsz = 128  # 設定批次大小
# 訓練模型 開始進行訓練
history = model.fit(train_images, train_labels,epochs=100, # 6萬張影像各訓練100次
                    batch_size = batchsz,   # 設定批次訓練大小
                    validation_split = 0.2,    # 劃分資料集的 20% 作為驗證集用
                    verbose = 2)  # 印出為精簡模式

# 確認模型預測結果

首先將測試資料的前十五筆資料印出並比對前十五筆資料的標籤

In [ ]:
import numpy as np
# 測試資料的預處理
test_image, test_labels = preprocess(test_image, test_label)
predicted_image15 = model.predict(test_image[:15])       # 只先丟前15張進去測試
predicted_ids15 = np.argmax(predicted_image15, axis=-1)  # argmax為取出機率最大的index，axis=-1 為最後資訊
print("Predicted labels: ", predicted_ids15[:15])
print("test labels: ", test_label[:15])

保存模型

In [ ]:
# 儲存網路
model.save('Fashion_model.h5') # 存在相同資料夾
print('Save Model')
del model                      # 從專案中刪除模型，之後進行測試

In [ ]:
# 載入模型
print('loaded model from Fashion_model.h5')
Model2 = tf.keras.models.load_model('Fashion_model.h5',compile=False) # 不做compile要設定為false(預設值為true)
# 拿前十五筆資料來預測並印出標籤
prediction = Model2.predict(test_image[:15])
print(tf.argmax(prediction,1))
# 印出前十五筆資料的正確標籤
print(test_labels[:15])

In [ ]:
predicted_image = Model2.predict(test_image)  # 進行預測
print(predicted_image.shape)
predicted_ids = np.argmax(predicted_image, axis=-1)  # 取出機率最大的 index
print("Predicted labels: ", predicted_ids)
print("test labels: ", test_label)
# 比較兩個 predicted_ids 與 test_label 是否相同
correct_prediction = tf.equal(predicted_ids,test_label)  
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
print(accuracy*100)

# 6

載入模型

In [ ]:
import tensorflow as tf

(train_image,train_label),(test_image,test_label)=\
    tf.keras.datasets.fashion_mnist.load_data()

# 對資料集做一個前置處理, 將資料正規到 0~1 之間
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x,y

# 載入模型
print('loaded model from Fashion_model.h5')
test_image, test_labels = preprocess(test_image, test_label)
# 不做compile要設定為false(預設值為true)
Model2 = tf.keras.models.load_model('Fashion_model.h5',compile=False)
# 拿前十五筆資料來預測並印出標籤
prediction = Model2.predict(test_image[:15])
print("Predicted labels:", tf.argmax(prediction,1))
# 印出前十五筆資料的正確標籤
print("test labels: ",test_labels[:15])

In [ ]:
predicted_image = Model2.predict(test_image)  # 進行預測
print(predicted_image.shape)
predicted_ids = np.argmax(predicted_image, axis=-1)  # 取出機率最大的 index
print("Predicted labels: ", predicted_ids)
print("test labels: ", test_label)
# 比較兩個 predicted_ids 與 test_label 是否相同
correct_prediction = tf.equal(predicted_ids,test_label)  
accuracy = tf.reduce_mean(tf.cast(correct_prediction,"float"))
print(accuracy*100)

繼續訓練模型

In [ ]:
import tensorflow as tf

(train_image,train_label),(test_image,test_label)=\
    tf.keras.datasets.fashion_mnist.load_data()

# 對資料集做一個前置處理, 將資料正規到 0~1 之間
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x,y

train_image, train_label = preprocess(train_image, train_label)

# 載入模型
print('loaded model from Fashion_model.h5')
ReloadModel = tf.keras.models.load_model('Fashion_model.h5')
batchsz = 128                              # 設定批次大小
ReloadModel.fit(train_image, train_label,epochs=10,
                batch_size = batchsz,      # 設定批次訓練大小
                validation_split = 0.2,    # 劃分資料集的 20% 作為驗證集用
                verbose = 2)

# 8

In [ ]:
import tensorflow as tf

(train_image,train_label),(test_image,test_label)=\
    tf.keras.datasets.fashion_mnist.load_data()
# 對資料集做一個前置處理, 將資料正規到 0~1 之間
def preprocess(x, y):
    x = tf.cast(x, dtype=tf.float32) / 255.
    y = tf.cast(y, dtype=tf.int32)
    return x,y
test_images, test_labels = preprocess(test_image, test_label)
# 載入模型
print('loaded model from Fashion_model.h5')
ReloadModel = tf.keras.models.load_model('Fashion_model.h5')
loss,accuracy = ReloadModel.evaluate(test_images,test_labels)
# 列印損失值與正確率
print("\n test loss : ", loss)
print("\n test accuracy : ", accuracy)

# 9

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
# 匯入Keras的mnist模組
from tensorflow.keras.datasets import mnist
(train_Data, train_Label), (test_Data, test_Label) = mnist.load_data()

model = Sequential([
    layers.Flatten(input_shape=(28, 28)),   # 將輸入資料從 28x28 攤平成 784
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # output 為 10 個 class
])

# model 每層定義好後需要經過 compile
# sparse_categorical_crossentropy 的標籤是 integer
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 將建立好的 model 去 fit 我們的 training data
model.fit(train_Data, train_Label, epochs=10)    # fit一定要拿訓練資料
# 利用 test_Data 去進行模型評估
# verbose = 2 為每個 epoch 輸出一行紀錄
model.evaluate(test_Data, test_Label, verbose=2) # 評估一定要拿測試資料

# 模型權重保存
print("save_weights")
model.save_weights('modelWeight.h5')
del model

# 重新定義模型結構
model_2 = Sequential([
    layers.Flatten(input_shape=(28, 28)),   # 將輸入資料從 28x28 攤平成 784
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # output 為 10 個 class
])
# 雖然得到權重，但還是要再加入優化函式、損失函式，才能進行評估
model_2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model_2.load_weights('modelWeight.h5')
loss, accuracy = model_2.evaluate(test_Data, test_Label)
print("\n test loss:",loss)
print("\n test accuracy:",accuracy)

# 10

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
# 匯入Keras的mnist模組
from tensorflow.keras.datasets import mnist
(train_Data, train_Label), (test_Data, test_Label) = mnist.load_data()

model = Sequential([
    layers.Flatten(input_shape=(28, 28)),   # 將輸入資料從 28x28 攤平成 784
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # output 為 10 個 class
])

# model 每層定義好後需要經過 compile
# sparse_categorical_crossentropy 的標籤是 integer
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 將建立好的 model 去 fit 我們的 training data
model.fit(train_Data, train_Label, epochs=10)
# 利用 test_Data 去進行模型評估
# verbose = 2 為每個 epoch 輸出一行紀錄
model.evaluate(test_Data, test_Label, verbose=2)

In [ ]:
import tensorflow as tf
# 儲存網路至工作路徑底下的 temp 資料夾
tf.saved_model.save(model,'temp')
print('Save Model')
# 刪除模型
del model

In [ ]:
print('Load Model')
# 從 temp 資料夾底下載入模型
# ModelNew = tf.saved_model.load('temp')   # 有問題
ModelNew = tf.keras.models.load_model('temp')
categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
# 輸入測試資料
y_pred = ModelNew(test_Data)

# 利用 update_state() 設定更新真實值與預測值的數據
categorical_accuracy.update_state(y_true=test_Label, y_pred=y_pred)
# 執行結果
print("Test Accuracy : ",categorical_accuracy.result())

# 11

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
# 匯入Keras的mnist模組
from tensorflow.keras.datasets import mnist
(train_Data, train_Label), (test_Data, test_Label) = mnist.load_data()

model = Sequential([
    layers.Flatten(input_shape=(28, 28)),   # 將輸入資料從 28x28 攤平成 784
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # output 為 10 個 class
])

# model 每層定義好後需要經過 compile
# sparse_categorical_crossentropy 的標籤是 integer
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

from tensorflow.keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_accuracy:.4f}.h5',
                                  monitor='val_accuracy',save_best_only=True,verbose=1)

# 將建立好的 model 去 fit 我們的 training data
history = model.fit(train_Data, train_Label, 
                    epochs=20, batch_size=512,
                    validation_split=0.2,verbose=2,
                    callbacks=[model_checkpoint])  # 添加 callback
# 利用 test_Data 去進行模型評估
# verbose = 2 為每個 epoch 輸出一行紀錄
model.evaluate(test_Data, test_Label, verbose=2)

# 12

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
# 匯入Keras的mnist模組
from tensorflow.keras.datasets import mnist
(train_Data, train_Label), (test_Data, test_Label) = mnist.load_data()

model = Sequential([
    layers.Flatten(input_shape=(28, 28)),   # 將輸入資料從 28x28 攤平成 784
    layers.Dense(256, activation='relu'),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')  # output 為 10 個 class
])

# model 每層定義好後需要經過 compile
# sparse_categorical_crossentropy 的標籤是 integer
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

early_stopping = EarlyStopping(monitor='val_accuracy', min_delta=0.01,
                               patience=0,verbose=1)
model_checkpoint = ModelCheckpoint(filepath='weights.{epoch:02d}-{val_accuracy:.4f}.h5',
                                  monitor='val_accuracy',save_best_only=True,verbose=1)

# 將建立好的 model 去 fit 我們的 training data
history = model.fit(train_Data, train_Label, 
                    epochs=20, batch_size=500,
                    validation_split=0.2,
                    callbacks=[early_stopping, model_checkpoint])  # 添加 callback
# 利用 test_Data 去進行模型評估
# verbose = 2 為每個 epoch 輸出一行紀錄
model.evaluate(test_Data, test_Label, verbose=2)